In [1]:
from azureml.core import Workspace, Experiment


project_title = "udacity-project"
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name=project_title)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-153519
Azure region: southcentralus
Subscription id: 5a4ab2ba-6c51-4805-8155-58759ad589d8
Resource group: aml-quickstarts-153519


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

cluster_name = "cluster-vhcg77"
# verify that the cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name = cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes = 1, idle_seconds_before_scaledown=120)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--solver': choice('newton-cg', 'lbfgs', 'liblinear'),
    #'--penalty': choice('none', 'l1', 'l2'),
    '--C': choice(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100),
    '--max_iter': choice(100, 200, 300, 400, 500)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=2, delay_evaluation=10)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./', compute_target=cpu_cluster, entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig( hyperparameter_sampling=ps, 
                                        primary_metric_name='Accuracy', 
                                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                        max_total_runs=100, 
                                        max_concurrent_runs=4, 
                                        max_duration_minutes=10080, 
                                        policy=policy, 
                                        estimator=est)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
assert(hyperdrive_run.get_status() == 'Completed')

In [7]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best parameters: ', parameter_values)
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])


best_model = best_run.register_model(model_name='udacity_project_model', model_path='./')

Best parameters:  ['--C', '10', '--max_iter', '100', '--solver', 'newton-cg']
Best Run Id:  HD_31520294-36e6-4719-8797-9bc05eb84aa5_83

 Accuracy: 0.9100151745068286


In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset = TabularDatasetFactory.from_delimited_files(dataset_path)

In [11]:
from train import clean_data
import pandas as pd
from sklearn.model_selection import train_test_split
from azureml.data.dataset_factory import TabularDatasetFactory
import os

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)

cleaned_dataset = pd.concat([x,y], axis=1)

# Split data into train and test data
train_dataset, test_dataset = train_test_split(cleaned_dataset, test_size=0.2, random_state=42)

directory = 'train'
if not os.path.exists(directory):
    os.makedirs(directory)
    
directory = 'test'
if not os.path.exists(directory):
    os.makedirs(directory)

# Export data as csv
train_dataset.to_csv("train/train_data.csv", index=False)
test_dataset.to_csv("test/test_data.csv", index=False)

# Upload data to the datastore
datastore = ws.get_default_datastore()
datastore.upload(src_dir='./train', target_path = project_title)
datastore.upload(src_dir='./test', target_path = project_title)


train_dataset_azure = TabularDatasetFactory.from_delimited_files(path=datastore.path("udacity-project/train_data.csv"))
test_dataset_azure = TabularDatasetFactory.from_delimited_files(path=datastore.path("udacity-project/test_data.csv"))



Uploading an estimated of 1 files
Uploading ./train/train_data.csv
Uploaded ./train/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Uploading an estimated of 1 files
Uploading ./test/test_data.csv
Uploaded ./test/test_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [19]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_config = AutoMLConfig(
                                compute_target=cpu_cluster,
                                task='classification',
                                training_data=train_dataset_azure,
                                label_column_name='y',
                                n_cross_validations=5,
                                primary_metric='accuracy',
                                experiment_timeout_minutes=30,
                                max_concurrent_iterations=4,
                                max_cores_per_iteration=-1)

In [20]:
# Submit your automl run

run = exp.submit(config=automl_config, show_output=True)
run.wait_for_completion()

Submitting remote run.
No run_configuration provided, running on cluster-vhcg77 with default configuration
Running on remote compute: cluster-vhcg77


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_161e0248-6513-4278-96be-639705f5423b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+-------------------------

        34    StackEnsemble                                 0:01:18       0.9156    0.9159
        33    VotingEnsemble                                0:01:06       0.9174    0.9174


{'runId': 'AutoML_161e0248-6513-4278-96be-639705f5423b',
 'target': 'cluster-vhcg77',
 'status': 'Completed',
 'startTimeUtc': '2021-08-03T21:30:34.901168Z',
 'endTimeUtc': '2021-08-03T22:16:56.334623Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cluster-vhcg77',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"5a4ab2ba-6c51-4805-8155-58759ad589d8","resource_group":"aml-quickstarts-153519","workspace_name":"quick-starts-ws-153519","region":"southcentralus","compute_target":"cluster-vhcg77","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metric":"a

In [22]:
# Retrieve and save your best automl model.

best_automl_run, fitted_automl_model = run.get_output()
print(best_automl_run)

Run(Experiment: udacity-project,
Id: AutoML_161e0248-6513-4278-96be-639705f5423b_33,
Type: azureml.scriptrun,
Status: Completed)


In [26]:
print("Best run metrics: ")
best_automl_run.get_metrics()

Best run metrics: 


{'matthews_correlation': 0.5503703359076677,
 'precision_score_micro': 0.9173748103186646,
 'precision_score_weighted': 0.9114272556373315,
 'accuracy': 0.9173748103186646,
 'weighted_accuracy': 0.9588648681145153,
 'AUC_weighted': 0.9485751865212653,
 'recall_score_weighted': 0.9173748103186646,
 'balanced_accuracy': 0.7503022688553409,
 'log_loss': 0.18149601406418667,
 'AUC_micro': 0.9811092501168599,
 'norm_macro_recall': 0.5006045377106819,
 'recall_score_macro': 0.7503022688553409,
 'f1_score_micro': 0.9173748103186646,
 'precision_score_macro': 0.8027588994046081,
 'f1_score_weighted': 0.9134446484360115,
 'average_precision_score_macro': 0.8279229626388267,
 'average_precision_score_weighted': 0.956322316282282,
 'f1_score_macro': 0.7727874663364912,
 'average_precision_score_micro': 0.9819124289533295,
 'recall_score_micro': 0.9173748103186646,
 'AUC_macro': 0.9485751865212653,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_161e0248-6513-4278-96be-639705f5423b_

In [27]:
fitted_automl_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=None, reg_alpha=0.5789473684210527, reg_lambda=0.631578947368421, subsample=0.6436842105263159))], verbose=False))], flatten_transform=None, weights=[0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.2222222222222222, 0.2222222222222222, 0.1111111111111111]))],
         verbose=False)

In [41]:
#print(best_automl_run.get_file_names())

models = [element for element in best_automl_run.get_file_names() if 'pkl' in element]
models
for model in models:
    best_automl_run.download_file(name=model)

In [43]:
cpu_cluster.delete()

KeyError: 'azure-asyncoperation'